In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence

class UTKFaceDataGenerator(Sequence):
    def __init__(self, img_folder, img_list, labels, batch_size=32, img_size=(224, 224), shuffle=True):
        self.img_folder = img_folder
        self.img_list = img_list
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.img_list) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_img_list = [self.img_list[k] for k in indices]
        batch_labels = [self.labels[k] for k in indices]

        X, y = self.__data_generation(batch_img_list, batch_labels)
        return X, y

    def on_epoch_end(self):
        self.indices = np.arange(len(self.img_list))
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __data_generation(self, batch_img_list, batch_labels):
        X = np.empty((self.batch_size, *self.img_size, 3))
        y = np.empty((self.batch_size), dtype=int)

        for i, img_name in enumerate(batch_img_list):
            img = cv2.imread(os.path.join(self.img_folder, img_name))
            img = cv2.resize(img, self.img_size)
            img = img / 255.0
            X[i,] = img
            y[i] = batch_labels[i]

        return X, y

# Define paths
data_path = 'utk/utkcropped/utkcropped'  # Update this path

# Prepare data lists
img_list = []
ages = []

# Load image names and labels
for i, img_name in enumerate(os.listdir(data_path)):
    if i >= 10000:
        break
    img_list.append(img_name)
    age = int(img_name.split('_')[0])
    ages.append(age)

# Split data into training and testing
from sklearn.model_selection import train_test_split
train_img_list, test_img_list, train_labels, test_labels = train_test_split(img_list, ages, test_size=0.2, random_state=42)

# Create data generators
batch_size = 32
train_generator = UTKFaceDataGenerator(data_path, train_img_list, train_labels, batch_size=batch_size)
test_generator = UTKFaceDataGenerator(data_path, test_img_list, test_labels, batch_size=batch_size, shuffle=False)


In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Multiply, Add, Activation, BatchNormalization

# Define the attention block
def attention_block(inputs, filters):
    g1 = Conv2D(filters, (1, 1), padding='same')(inputs)
    g1 = BatchNormalization()(g1)
    g1 = Activation('relu')(g1)

    g2 = Conv2D(filters, (3, 3), padding='same')(inputs)
    g2 = BatchNormalization()(g2)
    g2 = Activation('relu')(g2)

    g3 = Conv2D(filters, (5, 5), padding='same')(inputs)
    g3 = BatchNormalization()(g3)
    g3 = Activation('relu')(g3)

    attn = Add()([g1, g2, g3])
    attn = Conv2D(1, (1, 1), padding='valid', activation='sigmoid')(attn)
    attn = Multiply()([inputs, attn])

    return attn

# Build the model
input_shape = (224, 224, 3)
inputs = Input(shape=input_shape)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = MaxPooling2D((2, 2))(x)
x = attention_block(x, 32)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = attention_block(x, 64)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = attention_block(x, 128)

x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(1)(x)  # For regression (age prediction)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.summary()


2024-05-21 14:42:38.885639: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-21 14:42:38.919259: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-21 14:42:38.919495: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 224, 224, 32)         896       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)         0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 112, 112, 32)         1056      ['max_pooling2d[0][0]']   

In [5]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(train_generator, epochs=20, validation_data=test_generator, callbacks=[early_stopping])


Epoch 1/20


2024-05-21 14:42:46.001937: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-05-21 14:42:50.472644: I external/local_xla/xla/service/service.cc:168] XLA service 0x872bd60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-21 14:42:50.472665: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-05-21 14:42:50.475948: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1716282770.538213   37503 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


250/250 [==============================] - 61s 163ms/step - loss: 416.0507 - mae: 15.4607 - val_loss: 397.0396 - val_mae: 16.0450
Epoch 2/20
250/250 [==============================] - 37s 149ms/step - loss: 285.5162 - mae: 13.0588 - val_loss: 1264.7233 - val_mae: 30.8093
Epoch 3/20
250/250 [==============================] - 37s 150ms/step - loss: 255.1515 - mae: 12.3448 - val_loss: 647.1624 - val_mae: 22.0897
Epoch 4/20
250/250 [==============================] - 37s 150ms/step - loss: 233.8052 - mae: 11.7371 - val_loss: 270.0988 - val_mae: 12.7434
Epoch 5/20
250/250 [==============================] - 38s 150ms/step - loss: 219.8386 - mae: 11.4061 - val_loss: 621.9278 - val_mae: 19.2196
Epoch 6/20
250/250 [==============================] - 38s 151ms/step - loss: 215.2880 - mae: 11.2743 - val_loss: 1091.2350 - val_mae: 27.9640
Epoch 7/20
250/250 [==============================] - 38s 151ms/step - loss: 219.0521 - mae: 11.3203 - val_loss: 938.4686 - val_mae: 24.6856
Epoch 8/20
250/250 [==

In [6]:
test_loss, test_mae = model.evaluate(test_generator)
print(f'Test Mean Absolute Error: {test_mae}')


62/62 [==============================] - 3s 45ms/step - loss: 270.0988 - mae: 12.7434
Test Mean Absolute Error: 12.743355751037598


In [7]:
model.save('age_detection_model_5000_images.h5')


/home/eragon/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
import tensorflow as tf

In [5]:
model = load_model('age_detection_model_5000_images.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


with open('age_detection_model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpvhbeuuom/assets


INFO:tensorflow:Assets written to: /tmp/tmpvhbeuuom/assets
2024-05-21 16:57:59.609924: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-05-21 16:57:59.609947: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-05-21 16:57:59.610364: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpvhbeuuom
2024-05-21 16:57:59.613959: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-05-21 16:57:59.613972: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpvhbeuuom
2024-05-21 16:57:59.623529: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-05-21 16:57:59.626935: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-05-21 16:57:59.769000: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [6]:
import numpy as np
import tensorflow as tf
import cv2

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path='age_detection_model.tflite')
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [10]:
def preprocess_image(image):
    img = cv2.resize(image, (224, 224))
    img = img.astype(np.float32)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

FOR FACE DETECTION/TRACKING

In [11]:
# Load the Haar Cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [13]:
# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) == 0:
        # No face detected, predict age as 0
        age_prediction = 0
    else:
        for (x, y, w, h) in faces:
            # Draw a red rectangle around the detected face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

            # Preprocess the face region
            face = frame[y:y+h, x:x+w]
            input_data = preprocess_image(face)

            # Set the tensor to point to the input data to be inferred
            interpreter.set_tensor(input_details[0]['index'], input_data)

            # Run the interpreter
            interpreter.invoke()

            # Get the predicted age
            output_data = interpreter.get_tensor(output_details[0]['index'])
            age_prediction = output_data[0][0]

            # Display the results
            cv2.putText(frame, f'Predicted Age: {int(age_prediction)}', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Show the frame with the face rectangle and age prediction
    cv2.imshow('Age Detection', frame)

    # Press 'q' to quit the video stream
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture and close windows
cap.release()
cv2.destroyAllWindows()


QObject::moveToThread: Current thread (0x1193b860) is not the object's thread (0x16651b90).
Cannot move to target thread (0x1193b860)

QObject::moveToThread: Current thread (0x1193b860) is not the object's thread (0x16651b90).
Cannot move to target thread (0x1193b860)

QObject::moveToThread: Current thread (0x1193b860) is not the object's thread (0x16651b90).
Cannot move to target thread (0x1193b860)

QObject::moveToThread: Current thread (0x1193b860) is not the object's thread (0x16651b90).
Cannot move to target thread (0x1193b860)

QObject::moveToThread: Current thread (0x1193b860) is not the object's thread (0x16651b90).
Cannot move to target thread (0x1193b860)

QObject::moveToThread: Current thread (0x1193b860) is not the object's thread (0x16651b90).
Cannot move to target thread (0x1193b860)

QObject::moveToThread: Current thread (0x1193b860) is not the object's thread (0x16651b90).
Cannot move to target thread (0x1193b860)

QObject::moveToThread: Current thread (0x1193b860) is n

KeyboardInterrupt: 